# **Tâche #2 - Questions-réponses avec un modèle QA extractif**

Cette tâche consiste à utiliser un modèle de question-réponse extractif de type transformer afin de repérer des informations dans un texte. Vous utilisez la librairie HuggingFace pour accomplir cette tâche. On demande plus spécifiquement d’utiliser le modèle *bert-large-uncased-whole-word-masking-finetuned-squad*.

La tâche a pour but précis de repérer 3 informations dans les descriptions textuelles : le lieu et la date de l’incident ainsi qu’un court passage de texte indiquant ce qui s’est passé.  Une partie importante de votre travail consiste à trouver de bonnes formulations de questions pour repérer ces informations. Le fichier *t2_qa_examples*.json, qui contient 25 exemples annotés par un humain, est disponible pour mener vos expérimentations.

Les consignes pour cette tâche sont:
-	Nom du notebook : *t2_qa.ipynb* (ce notebook)
-	Tokenisation et plongements de mots : Ceux du modèle utilisé.
-	Normalisation : Aucune normalisation à faire (le tokeniseur convertit les lettres en minuscule).
-	Construction du modèle : vous utilisez la version préentraînée du modèle sans modification. Aucun affinement (fine-tuning) du modèle n’est requis pour cette tâche.
-	Évaluation : Du code est disponible dans le notebook pour évaluer la performance du modèle avec les métriques *exact match* et *F1*.
-	Analyse : Présentez et discutez des résultats que vous obtenez pour les 3 types d’informations à repérer. Discutez également de vos choix de questions pour accomplir cette tâche et les erreurs commises par le modèle QA.

Vous pouvez ajouter au notebook toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code.
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analysez vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc.
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Le chargement des données

Utilisez le fichier ***/data/t2_qa_examples.json*** pour mener vos expérimentations. 

In [20]:
import json

def load_json_data(filename):
    with open(filename, 'r') as fp:
        data = json.load(fp)
    return data

In [21]:
# Charger et afficher quelques exemples
from pprint import pprint

data = load_json_data('../data/t2_qa_examples.json')
print("Nombre total d'exemples:", len(data))

# utilisation de pprint pour afficher 5 exemplses


pprint(data[:5])

Nombre total d'exemples: 25
[{'EVENT': 'Employee #1  was struck and thrown',
  'WHEN': 'November 10  2013',
  'WHERE': 'railroad bridge overpass',
  'text': ' At around 10:00 p.m. on November 10  2013  Employee #1  with '
          'Villager  Construction Inc.  with a coworker  were using an asphalt '
          'milling machine  (Wirtgen; Model Number: W2100) to grind out '
          'existing asphalt from an  interstate at a railroad bridge overpass. '
          'Employee # 1 was standing on the  ground  checking the depth of the '
          'cut into the asphalt  using a handheld  pendant attached to the '
          'machine. The pedant could stretch out from ten to 15  ft. This '
          'allowed Employee #1 to walk back and forth  checking the cut. The  '
          'operator was on the top of the milling machine  controlling the '
          'operation of  the machine and ensuring that the milling machine and '
          'dump truck (driven by a  second coworker  who worked for an

## 2. Vos questions 

Vous pouvez mettre plusieurs options de questions dans le notebook. Il est important de présenter, au minimum, les résultats pour le meilleur jeu de questions. Vous pourrez également mettre des informations à ce propos dans la section d'analyse. 

In [22]:
# Exemple de questions pour les 3 types d'informations

questions = {
    "WHEN": [
        "What is the exact date and time when the incident occurred?",
    ],
    "WHERE": [  
        "Where did the event occur ? ",
    ],
    "EVENT": [
        "What unfolded during the incident?",
    ],
}




## 3. Le modèle de question-réponse extractif

In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch

# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")



Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# Tronquer le texte à 512 tokens

def answer_question(question, context):
    # Tronquer le contexte si nécessaire
    max_length = 512
    if len(context) > max_length:
        context = context[:max_length]

    inputs = tokenizer(question, context, return_tensors='pt', truncation="only_second", padding="max_length", max_length=512, stride=128)
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1  # +1 pour inclure le dernier token
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer


In [25]:
# Questions pour extraire des informations
context = data[0]['text']

event = answer_question("What happened to Employee # 1?", context)
location = answer_question("Where did the incident take place?", context)
date = answer_question("When did the incident occur?", context)

print("Contexte:", context)
print("Event:", event)
print("Location:", location)
print("Date:", date)


Contexte:  At around 10:00 p.m. on November 10  2013  Employee #1  with Villager  Construction Inc.  with a coworker  were using an asphalt milling machine  (Wirtgen; Model Number: W2100) to grind out existing asphalt from an  interstate at a railroad bridge overpass. Employee # 1 was standing on the  ground  checking the depth of the cut into the asphalt  using a handheld  pendant attached to the machine. The pedant could stretch out from ten to 15  ft. This allowed Employee #1 to walk back and forth  checking the cut. The  operator was on the top of the milling machine  controlling the operation of  the machine and ensuring that the milling machine and dump truck (driven by a  second coworker  who worked for an independent trucking service) kept a safe  working distance. A different company  Protective Services Inc. (PSI)  was  responsible for the traffic control of the job site and had shut down the  inside lane of a three lane section of the interstate  so that work could be  condu

## 4. Des fonctions utilitaires pour l'évaluation

In [26]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [27]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if len(ground_truth_tokens) == 0 or len(prediction_tokens) == 0:
        return int(ground_truth_tokens == prediction_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def evaluate_exact_match(ground_truth, prediction):
    """Vérifie si les 2 textes sont quasi-identiques."""
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

## 5. Évaluation du modèle et analyse

In [28]:
# Définir un seuil pour les scores faibles (par exemple, 0.5)
low_score_threshold = 0.5

# Initialiser une liste pour stocker les exemples ayant des scores faibles
low_score_examples = {"WHEN": [], "EVENT": [], "WHERE": []}

# Initialiser les résultats d'évaluation
evaluation_results = {
    "WHEN": [],
    "WHERE": [],
    "EVENT": []
}

# Exemple d'évaluation
for example in data:
    # Vérification du type de texte
    text = example["text"]  # Doit être une chaîne
    if not isinstance(text, str):
        print(f"Erreur : Le texte doit être une chaîne. Type trouvé : {type(text)}")
        continue  # Passer à l'exemple suivant si le type est incorrect

    ground_truths = {
        "WHEN": example["WHEN"],  # Réponse correcte pour WHEN
        "WHERE": example["WHERE"],  # Réponse correcte pour WHERE
        "EVENT": example["EVENT"]   # Réponse correcte pour EVENT
    }
    
    # Appel à la fonction de prédiction
    predictions = {}
    for key in questions.keys():
        # Nous utilisons la question correspondante pour chaque catégorie
        question = questions[key][0]  # Prenons la première question pour simplifier
        predictions[key] = answer_question(question, text)  # Assurez-vous que le texte est bien une chaîne
    
    # Évaluation pour chaque catégorie
    for key in ["WHEN", "WHERE", "EVENT"]:
        f1 = evaluate_f1(ground_truths[key], predictions[key])  # Calcul du score F1
        em = evaluate_exact_match(ground_truths[key], predictions[key])  # Calcul de l'Exact Match
        print(f"{key} - F1: {f1:.2f}, Exact Match: {em:.2f}")

        # Stocker les résultats dans evaluation_results
        evaluation_results[key].append(f1)  # Stocker le score F1

        if f1 < low_score_threshold:
            # Ajouter l'exemple et les informations de comparaison à low_score_examples
            low_score_examples[key].append({
                "text": text,
                "expected_answer": ground_truths[key],
                "predicted_answer": predictions[key],
                "f1_score": f1
            })

WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.33, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.09, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.20, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 0.29, Exact Match: 0.00
EVENT - F1: 0.00, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.32, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.47, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.88, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 1.00, Exact Match: 1.00
EVENT - F1: 0.00, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WHERE - F1: 0.80, Exact Match: 0.00
EVENT - F1: 0.00, Exact Match: 0.00
WHEN - F1: 1.00, Exact Match: 1.00
WH

In [29]:
# Initialiser les compteurs
success_count = {}
failure_count = {}
total_count = {}

# Calculer le nombre de réussites et d'échecs pour chaque catégorie
for category, scores in evaluation_results.items():
    total_count[category] = len(scores)  # Nombre total d'exemples
    success_count[category] = sum(1 for score in scores if score > 0.5)  # Considérer comme succès si score > 0.5
    failure_count[category] = total_count[category] - success_count[category]  # Échecs

# Calculer les pourcentages
success_percentage = {category: (count / total_count[category]) * 100 for category, count in success_count.items()}
failure_percentage = {category: (count / total_count[category]) * 100 for category, count in failure_count.items()}

# Affichage des résultats
for category in total_count.keys():
    print(f"{category} - Success: {success_percentage[category]:.2f}%, Failure: {failure_percentage[category]:.2f}%")


WHEN - Success: 96.00%, Failure: 4.00%
WHERE - Success: 76.00%, Failure: 24.00%
EVENT - Success: 40.00%, Failure: 60.00%


In [30]:
# Afficher les exemples avec scores faibles pour analyse
for category, examples in low_score_examples.items():
    print(f"\n--- {category} - Exemples avec scores faibles ---")
    for i, example in enumerate(examples, 1):
        print(f"Exemple #{i}")
        print("Texte :", example["text"])
        print("Réponse attendue :", example["expected_answer"])
        print("Réponse générée :", example["predicted_answer"])
        print(f"Score F1 : {example['f1_score']:.2f}\n")



--- WHEN - Exemples avec scores faibles ---
Exemple #1
Texte :  Employee #1  a diver  became caught in a coffer dam and drowned.                
Réponse attendue : 
Réponse générée : drowned
Score F1 : 0.00


--- EVENT - Exemples avec scores faibles ---
Exemple #1
Texte :  At around 10:00 p.m. on November 10  2013  Employee #1  with Villager  Construction Inc.  with a coworker  were using an asphalt milling machine  (Wirtgen; Model Number: W2100) to grind out existing asphalt from an  interstate at a railroad bridge overpass. Employee # 1 was standing on the  ground  checking the depth of the cut into the asphalt  using a handheld  pendant attached to the machine. The pedant could stretch out from ten to 15  ft. This allowed Employee #1 to walk back and forth  checking the cut. The  operator was on the top of the milling machine  controlling the operation of  the machine and ensuring that the milling machine and dump truck (driven by a  second coworker  who worked for an independent t